In [1]:
from pyscf import gto,scf
import numpy as np
import matplotlib.pyplot as plt
from FcMole import *
from AP_utils import DeltaV
from numpy.linalg import norm
from alch_deriv import alchemy_cphf_deriv,make_dP

In [2]:
CO=gto.M(atom="C 0 0 0; O 0 0 2 ",unit="Bohr", basis="def2-qzvp")

In [3]:
mf=scf.RKS(CO)
mf.xc="PBE0"
mf.scf()
P0=mf.make_rdm1()
C0=mf.mo_coeff

converged SCF energy = -113.225870191075


In [4]:
CO1=FcM_like(CO,fcs=[.5,0])
mf1=scf.RKS(CO1)
mf1.xc="PBE0"
mf1.scf(dm0=P0)
P1=mf1.make_rdm1()
C1=mf1.mo_coeff

/home/giorgio/anaconda3/lib/python3.8/site-packages/pyscf/gto/mole.py:1209: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -120.958557196315


In [5]:
dV=DeltaV(CO,[[0],[.1]])

In [6]:
Delt_e_elec=mf1.energy_elec()[0]-mf.energy_elec()[0]
np.einsum("ij,ij",P0,dV),Delt_e_elec  # APDFT1 predictions 

(-1.87147721825293, -9.732687005239313)

In [7]:
from pyscf.scf import cphf
from pyscf.scf import  _response_functions

In [9]:
rf=mf.gen_response(mf.mo_coeff, mf.mo_occ, hermi=1)
dm1test=np.ones_like(P0)/100
dm1test-=np.sum(dm1test*P0)/norm(P0)**2*P0
respfun=rf(dm1test)
dFt=mf.get_fock(dm=P0+dm1test)-mf.get_fock(dm=P0)
np.allclose(dFt,respfun),norm(dFt-respfun),norm(dFt)

(False, 0.3177638252603656, 25.17748281133913)

In [10]:
from pyscf.hessian.rhf import gen_vind
from pyscf.scf import cphf
from pyscf import lib

mo_energy,mo_coeff, mo_occ=mf.mo_energy,mf.mo_coeff, mf.mo_occ
vind = gen_vind(mf, mf.mo_coeff, mf.mo_occ)

In [11]:
mol=CO
int_r=DeltaV(CO,[[0],[.01]])
occidx = mo_occ > 0
orbo = mo_coeff[:, occidx]
orbv = mo_coeff[:,~occidx]
charges = mol.atom_charges()
coords  = mol.atom_coords()
charge_center = np.einsum('i,ix->x', charges, coords) / charges.sum()
h1 = lib.einsum('pq,pi,qj->ij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals
#h1=h1.reshape((1,h1.shape[0],h1.shape[1]))

In [12]:
h1.shape

(114, 7)

In [13]:
#mol = mf.mol
#mo_energy = mf.mo_energy
#mo_coeff = mf.mo_coeff
#mo_occ = mf.mo_occ
#occidx = mo_occ > 0
#orbo = mo_coeff[:, occidx]
#orbv = mo_coeff[:,~occidx]
#charges = mol.atom_charges()
#coords  = mol.atom_coords()
#charge_center = np.einsum('i,ix->x', charges, coords) / charges.sum()
#h1 = lib.einsum('pq,pi,qj->ij', int_r, mo_coeff.conj(), orbo) #going to molecular orbitals
#h1=h1.reshape((1,h1.shape[0],h1.shape[1]))
#s1 = np.zeros_like(h1)
#vind = gen_vind(mf, mo_coeff, mo_occ)
#mo1,e1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1,40, 1e-9)
#        s1=None
#mo1,e1 = cphf.solve(vind, mo_energy, mo_occ, h1, s1,10,1e-6)

In [14]:
mo1,e1=alchemy_cphf_deriv(mf,int_r)

In [15]:
dPfd=P1-P0

In [16]:
dP_ad=make_dP(mf,mo1)

In [17]:
norm(dPfd),norm(dPfd-dP_ad)

(0.8689826809050786, 0.8440748688867129)

In [18]:
from AP_class import APDFT_perturbator as AP

In [19]:
ap=AP(mf,sites=[0])
ap.build_all()

In [20]:
ap.APDFT1([.5]),mf1.e_tot

(-120.58325628234005, -120.95855719631467)

In [21]:
ap.APDFT2([.5]),mf1.e_tot

(-120.95525593774053, -120.95855719631467)

In [22]:
ap.APDFT3([.5]),mf1.e_tot

(-120.95909018384283, -120.95855719631467)

In [23]:
ap.af(0)/2

libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WA

array([[ 2.52953643e-15,  1.18568792e-15, -1.30949875e-01],
       [-2.80535782e-15, -2.15354674e-15,  1.22304874e-01]])

In [24]:
from pyscf.grad import rks


In [25]:
mf1.Gradients().grad()-mf.Gradients().grad()

libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- RKS gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.0798953447
1 O    -0.0000000000    -0.0000000000    -0.0798993333
----------------------------------------------


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- RKS gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.1968481851
1 O    -0.0000000000    -0.0000000000    -0.1968551477
----------------------------------------------


array([[ 6.63219236e-15,  3.79746054e-15, -1.16952840e-01],
       [-6.27847118e-15, -3.44705752e-15,  1.16955814e-01]])

In [26]:
from pyscf.geomopt import optimize

In [30]:
optimize(mf1,trustradius=1e-10)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   O   0.000000   0.000000   1.058354    0.000000  0.000000  0.000000
converged SCF energy = -120.958557196612


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.38183e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.26405e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.25595e+36


--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     0.0799016873
1 O     0.0000000000     0.0000000000   -28.5380588308
----------------------------------------------
cycle 1: E = -120.958557197  dE = -120.959  norm(grad) = 28.5382

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000  -0.079377    0.000000  0.000000 -0.079377
   O   0.000000   0.000000   1.137731    0.000000  0.000000  0.079377

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.



libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=1.0744e+37
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.37084e+36


converged SCF energy = -120.923186379274


libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=8.46374e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=1.30434e+37
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=7.93534e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=1.0744e+37
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=7.14469e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS NOT FOUND FOR POLYNOMIAL OF ORDER 4
rys_roots fails: nroots=6 x=4.4325e+36
libcint::rys_roots negative value in sqrt for roots 6 (j=5)
ROOT NUMBER 1 WAS 

--------------- SCF_Scanner gradients ---------------
         x                y                z
0 C    -0.0000000000     0.0000000000   168.6572428562
1 O    -0.0000000000     0.0000000000   186.0672980866
----------------------------------------------
cycle 2: E = -120.923186379  dE = 0.0353708  norm(grad) = 251.13

Geometry optimization cycle 3
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   C   0.000000   0.000000  -0.049304    0.000000  0.000000  0.030073
   O   0.000000   0.000000   1.107658    0.000000  0.000000 -0.030073

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess density matrix may have large error.



KeyboardInterrupt: 

In [333]:
mf1.scf(dm0=P1)

converged SCF energy = -120.958557203731


-120.95855720373132